## PATH SETUP

In [1]:
# MUST be first cell: set multiprocessing method for Windows
import torch.multiprocessing as mp
try:
    mp.set_start_method("spawn", force=True)
except RuntimeError:
    pass  # Already set

In [2]:
import os
import sys
from pathlib import Path

# FOR LOCAL USE THIS LINES
# current = Path.cwd()
# src_path = current / "src" if (current / "src").exists() else current.parent

# FOR COLAB USE THIS LINE INSTEAD
# BRANCH_NAME = "weighted_loss"  # Change this to switch branches
# !git clone -b {BRANCH_NAME} https://github.com/MatteoCamillo-code/GeoLoc-CVCS.git
# !cd /content/GeoLoc-CVCS && git pull origin {BRANCH_NAME} && cd ..
src_path = Path("/content/GeoLoc-CVCS/src").resolve()

sys.path.insert(0, str(src_path))

from utils.paths import find_project_root

# Set working directory and sys.path properly
project_root = find_project_root(src_path)
data_dir = project_root / "data"
history_dir = project_root / "outputs" / "history"
os.chdir(project_root)
sys.path.insert(0, str(project_root / "src"))
print("CWD:", Path.cwd())

CWD: /content/GeoLoc-CVCS


## IMPORT

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

from configs.baseline_multi_head import TrainConfig

cfg = TrainConfig()
if cfg.backbone == "resnet50":
    from torchvision.models import resnet50, ResNet50_Weights
elif cfg.backbone == "inceptionv4":
    import timm

from utils.seed import seed_everything
from utils.metrics import overall_val_acc_from_history
from utils.io import save_json, delete_file
from utils.paths import get_next_version, get_current_version
from training.runner import fit
from training.losses import CrossEntropyWithLabelSmoothing, compute_loss_weights

from src.utils.logging import get_logger
from src.utils.paths import abs_path

from models.multi_head_classifier_cbam import MultiHeadClassifier


In [4]:
seed_everything(cfg.seed)

device = cfg.device if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [5]:
import kagglehub

path = kagglehub.dataset_download("josht000/osv-mini-129k")
path = path + "/osv5m"
print("Path to dataset files:", path)

image_root = path + "/train_images"


Path to dataset files: /root/.cache/kagglehub/datasets/josht000/osv-mini-129k/versions/1/osv5m


In [6]:
if len(cfg.scenes) == 1 and cfg.scenes[0] == "total":
    train_val_path = data_dir / "metadata/s2-geo-cells/train_val_split_geocells_total.csv"
else:
    train_val_path = data_dir / "metadata/s2-geo-cells/train_val_split_geocells.csv"
cell_centers_dfs = {}
cells_hierarchy_dfs = {}

for scene in cfg.scenes:
    if cfg.same_partitions:
        cell_centers_path = data_dir / f"metadata/s2-geo-cells/cell_center_dataset_total.csv"
        cells_hierarchy_path = data_dir / f"metadata/s2-geo-cells/cell_hierarchy_dataset_total.csv"
    else:
        cell_centers_path = data_dir / f"metadata/s2-geo-cells/cell_center_dataset_{scene}.csv"
        cells_hierarchy_path = data_dir / f"metadata/s2-geo-cells/cell_hierarchy_dataset_{scene}.csv"

    cell_centers_df = pd.read_csv(cell_centers_path)
    cells_hierarchy_df = pd.read_csv(cells_hierarchy_path)

    # remove duplicates with same cell_id
    cell_centers_df = cell_centers_df.drop_duplicates(subset='cell_id_token')

    # Set S2 cell ID as index (assumes first column or 'cell_id' column)
    if 'cell_id_token' in cell_centers_df.columns:
        cell_centers_df = cell_centers_df.set_index('cell_id_token')
    else:
        # Set first column as index if it contains cell IDs
        cell_centers_df = cell_centers_df.set_index(cell_centers_df.columns[0])

    cell_centers_dfs[scene] = cell_centers_df
    cells_hierarchy_dfs[scene] = cells_hierarchy_df

## DATALOADER

In [7]:
from dataset.dataloader_utils import create_dataloaders

IMG_SIZE = 224

# Create all dataloaders with a single function call
loader_dict = create_dataloaders(
    image_root=image_root,
    csv_path=train_val_path,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    img_size=IMG_SIZE,
    seed=cfg.seed,
    train_subset_pct=cfg.train_size_pct,
    val_subset_pct=cfg.val_size_pct,
    scenes=cfg.scenes,
    augment=True,
    prefetch_factor=cfg.prefetch_factor,
    persistent_workers=True if cfg.num_workers > 0 else False,
    coarse_label_idx=cfg.coarse_label_idx,
)

## MODEL

In [8]:
!pip install s2sphere

In [10]:
models = {}
loss_weights = {}

for sc in cfg.scenes:

    if cfg.backbone == "inceptionv4":
        backbone = timm.create_model(
            'inception_v4',
            pretrained=True,
            features_only=True,
            out_indices=(3,)
        )
        FEAT_DIM = 1536

    elif cfg.backbone == "resnet50":
        resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        backbone = nn.Sequential(*list(resnet.children())[:-2])
        FEAT_DIM = 2048

    num_classes = list(map(
        lambda idx: len(loader_dict[sc]["label_maps"][f"label_config_{idx + 1}"]),
        cfg.coarse_label_idx
    ))

    loss_weights[sc] = compute_loss_weights(
        loader_dict[sc]["label_maps"], cfg.coarse_label_idx
    )

    model = MultiHeadClassifier(
        backbone=backbone.to(device),
        feat_dim=FEAT_DIM,
        head_dims=num_classes,
        dropout=cfg.dropout,
        coarse_level_idx=cfg.coarse_label_idx,
        use_cbam=cfg.use_cbam,
        cbam_reduction=cfg.cbam_reduction
    ).to(device)

    models[sc] = model
    print(f"Output classes {sc}:", num_classes)


Ciao baby
Output classes total: [4741, 2508, 1336]


In [11]:
criterion = CrossEntropyWithLabelSmoothing(ignore_index=-1, smoothing=cfg.label_smoothing)

# Create optimizer, scheduler, and scaler for each model
optimizers = {}
schedulers = {}
scalers = {}

for scene, model in models.items():
    optimizers[scene] = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    schedulers[scene] = StepLR(optimizers[scene], step_size=cfg.scheduler_step_size, gamma=cfg.scheduler_gamma)
    scalers[scene] = torch.amp.GradScaler(device=cfg.device, enabled=cfg.amp)

torch.backends.cudnn.benchmark = True
print(f"Initialized training components for {len(models)} scenes: {list(models.keys())}")

Initialized training components for 1 scenes: ['total']


## TRAINING LOOP

In [12]:
# Train each scene's model separately
histories = {}
logger = get_logger(log_file=str(abs_path(cfg.output_dir, "logs", "train.log")))
version = 0

base_name = f"{cfg.model_name}"
full_path = history_dir / f"{base_name}_v{version}.json"

for scene in models.keys():
    # Get versioned history path
    base_name = f"{cfg.model_name}_{scene}"
    version = get_next_version(history_dir, base_name)
    history_path = history_dir / f"{base_name}_v{version}.json"

    history = fit(
        cfg=cfg,
        model=models[scene],
        data_loader=loader_dict[scene],
        cell_centers=cell_centers_dfs[scene],
        cells_hierarchy=cells_hierarchy_dfs[scene],
        optimizer=optimizers[scene],
        criterion=criterion,
        loss_weights=loss_weights[scene],
        scaler=scalers[scene],
        use_tqdm=cfg.use_tqdm,
        scheduler=schedulers[scene],
        logger=logger,
        scene=scene,
        history_path=history_path,
        version=version,
    )

    histories[scene] = history

save_json(obj=histories, path=full_path)
for scene in cfg.scenes:
    base_name = f"{cfg.model_name}_{scene}"
    history_path = history_dir / f"{base_name}_v{version}.json"
    delete_file(history_path)



[18:35:15] INFO - Starting training MH_res_w_same_partitions for scene total ...


train:   0%|          | 0/788 [00:19<?, ?it/s]

val:   0%|          | 0/140 [00:18<?, ?it/s]

[18:39:20] INFO - Epoch 1/3 | train loss=7.1156 acc=13.33% | val loss=6.4059 acc=24.05% | geo acc={'acc@1km': '7.02%', 'acc@5km': '26.58%', 'acc@25km': '46.16%', 'acc@100km': '57.56%'} | time=244.4s


train:   0%|          | 0/788 [00:00<?, ?it/s]

val:   0%|          | 0/140 [00:00<?, ?it/s]

[18:42:37] INFO - Epoch 2/3 | train loss=5.8302 acc=35.70% | val loss=5.9659 acc=32.20% | geo acc={'acc@1km': '9.56%', 'acc@5km': '34.47%', 'acc@25km': '55.88%', 'acc@100km': '66.21%'} | time=196.6s


train:   0%|          | 0/788 [00:00<?, ?it/s]

val:   0%|          | 0/140 [00:00<?, ?it/s]

[18:45:56] INFO - Epoch 3/3 | train loss=5.3353 acc=47.25% | val loss=5.7592 acc=36.24% | geo acc={'acc@1km': '11.18%', 'acc@5km': '38.51%', 'acc@25km': '60.48%', 'acc@100km': '70.05%'} | time=198.1s
[18:45:56] INFO - Training completed.


In [14]:
# version = 3
accuracy_list = overall_val_acc_from_history(cfg, project_root, version)
print(accuracy_list)

FileNotFoundError: [Errno 2] No such file or directory: '/content/GeoLoc-CVCS/outputs/history/MH_res_w_same_partitions_total_v1.json'